In [51]:
# ロジスティック回帰とパーセプトロンの比較

require 'daru'
require 'nyaplot'
require 'numo/narray'

Variances = [5, 10, 30, 50] # 両クラス共通の分散（4種類の分散で計算を実施）

class Array
  def mean
    self.inject(:+) / self.size.to_f
  end
end

def normal_rand(mu = 0, sigma = 1.0)
  random = Random.new
  (Math.sqrt(-2 * Math.log(random.rand)) * Math.sin(2 * Math::PI * random.rand) * sigma) + mu
end

# データセット {x_n,y_n,type_n} を用意
def prepare_dataset(variance)
  n1 = 10
  n2 = 20
  mu1 = [7, 7]
  mu2 = [-3, -3]

  sigma = Math.sqrt(variance)

  df1 = n1.times.map do
    [normal_rand(mu1[0], sigma), normal_rand(mu1[1], sigma)]
  end
  df1 = df1.transpose
  df1 = Daru::DataFrame.new(x: df1[0], y: df1[1], type: Array.new(n1).fill(1))

  df2 = n2.times.map do
    [normal_rand(mu2[0], sigma), normal_rand(mu2[1], sigma)]
  end
  df2 = df2.transpose
  df2 = Daru::DataFrame.new(x: df2[0], y: df2[1], type: Array.new(n2).fill(0))

  df = df1.concat(df2)
  df = df.reindex(Daru::Index.new(df.index.to_a.shuffle))
  df[:index] = (n1 + n2).times.to_a
  df.set_index(:index)
end

# ロジスティック回帰
def run_logistic(train_set, plot)
  w = Numo::NArray[[0], [0.1], [0.1]]
  phi = train_set[:x, :y]
  phi[:bias] = Array.new(phi.size).fill(1)
  t = train_set[:type]
  t = t.to_matrix

  # 最大30回のIterationを実施
  30.times do
    # IRLS法によるパラメータの修正
    y = []
    phi.each_row do |line|
      a = Vector[*line.to_a].dot(w)
      y << (1.0 / (1.0 + Math.exp(-a)))
    end
    r = Matrix.diagonal(*(Numo::NArray[*y] * (1 - Numo::NArray[*y]).to_a))
    y = Numo::NArray[y].transpose
    tmp1 = Matrix[*Numo::NArray[*phi.to_matrix.transpose.to_a].dot(Numo::NArray[*r.to_a]).dot(Numo::NArray[*phi.to_matrix.to_a]).to_a].inverse
    tmp2 = Numo::NArray[*phi.to_matrix.transpose.to_a].dot(y - Numo::NArray[*t.transpose.to_a])
    w_new = w - Numo::NArray[*tmp1.to_a].dot(tmp2)

    # パラメータの変化が 0.1% 未満になったら終了
    if (w_new - w).transpose.dot(w_new - w).flatten[0] < (0.001 * (w.transpose.dot(w))).flatten[0]
      w = w_new
      break
    end
  end

  # 分類誤差の計算
  w0 = w[0]
  w1 = w[1]
  w2 = w[2]
  err = 0
  train_set.each_row do |point|
    x = point.x
    y = point.y
    type = point[:type]
    type = type * 2 - 1
    if type * (w0 + w1*x + w2*y) < 0
      err += 1
    end
  end

  err_rate = err * 100 / train_set.size

  # 結果を表示
  xmin = train_set.x.min - 5
  xmax = train_set.x.max + 10
  linex = Numo::NArray[*(xmin.to_i-5..xmax.to_i+4).to_a]
  liney = -linex * w1 / w2 - w0 / w2
  label = "ERR %.2f%%" % err_rate

  line_err = plot.add(:line, linex.cast_to(Numo::Int64).to_a, liney.cast_to(Numo::Int64).to_a)
  line_err.title(label)
  line_err.color('blue')
end

# パーセプトロン
def run_perceptron(train_set, plot)
  w0 = 0.0
  w1 = 0.0
  w2 = 0.0
  bias = 0.5 * (train_set.x.mean + train_set.y.mean)

  # Iterationを30回実施
  30.times do
    # 確率的勾配降下法によるパラメータの修正
    train_set.each_row do |point|
      x = point.x
      y = point.y
      type = point[:type]
      type = type * 2 - 1
      if type * (w0*bias + w1*x + w2*y) <= 0
        w0 += type * bias
        w1 += type * x
        w2 += type * y
      end
    end
  end

  # 分類誤差の計算
  err = 0
  train_set.each_row do |point|
    x = point.x
    y = point.y
    type = point[:type]
    type = type * 2 - 1
    if type * (w0*bias + w1*x + w2*y) <= 0
      err += 1
    end
  end

  err_rate = err * 100 / train_set.size

  # 結果を表示
  xmin = train_set.x.min - 5
  xmax = train_set.x.max + 10
  linex = Numo::NArray[*(xmin.to_i-5..xmax.to_i+4).to_a]
  liney = -linex * w1 / w2 - bias * w0 / w2
  label = "ERR %.2f%%" % err_rate

  line_err = plot.add(:line, linex.cast_to(Numo::Int64).to_a, liney.cast_to(Numo::Int64).to_a)
  line_err.title(label)
  line_err.color('red')
end

def run_simulation(variance, plot)
  train_set = prepare_dataset(variance)
  train_set1 = train_set.filter_rows {|row| row[:type] == 1 }
  train_set2 = train_set.filter_rows {|row| row[:type] == 0 }
  ymin = train_set.y.min - 5
  xmin = train_set.x.min - 5
  ymax = train_set.y.max + 10
  xmax = train_set.x.max + 10
  plot.configure do
    x_label("Variance: #{variance}")
    y_label('')
    xrange([xmin, xmax])
    yrange([ymin, ymax])
    legend(true)
    height(300)
    width(490)
  end

  scatter_true = plot.add_with_df(train_set1.to_nyaplotdf, :scatter, :x, :y)
  scatter_true.color('green')
  scatter_true.title('1')
  scatter_false = plot.add_with_df(train_set2.to_nyaplotdf, :scatter, :x, :y)
  scatter_false.color('orange')
  scatter_false.title('0')

  run_logistic(train_set, plot)
  run_perceptron(train_set, plot)
end

fig = Nyaplot::Frame.new

Variances.each do |variance|
  plot = Nyaplot::Plot.new
  run_simulation(variance, plot)
  fig.add(plot)
end

fig.show


<main>:6: warning: already initialized constant Variances
<main>:6: warning: previous definition of Variances was here


#<Nyaplot::Frame:0x007fef4d183fa0 @properties={:panes=>[#<Nyaplot::Plot:0x007fef4d183eb0 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fef4d5730e8 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :color=>"green", :title=>"1"}, :data=>"5704a6b6-29dc-495b-92c8-43cbeaeb3965"}, @xrange=[5.949976794453292, 9.852740813335013], @yrange=[4.1229305399737886, 12.940322853803211]>, #<Nyaplot::Diagram:0x007fef4d245bf0 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :color=>"orange", :title=>"0"}, :data=>"d630946c-e6af-4013-b4a2-3216e75ae4ed"}, @xrange=[-9.385755488192489, -0.15262523311360088], @yrange=[-5.205667661898412, 2.1657459349353783]>, #<Nyaplot::Diagram:0x007fef4d5472e0 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"ERR 0.00%", :color=>"blue"}, :data=>"277fb1b0-b78b-4dfc-99ed-da2907f45d3b"}, @xrange=[-19, 23], @yrange=[-23, 19]>, #<Nyaplot::Diagram:0x007fef4d455508 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"ERR 0.00%", :color=>"red"}, :data=>"7b205b13-5657-4a7b-be31-66e4434d2264"}, @xrange=[-19, 23], @yrange=[-26, 21]>], :options=>{:x_label=>"Variance: 5", :y_label=>"", :xrange=>[-14.385755488192489, 19.85274081333501], :yrange=>[-10.205667661898412, 22.94032285380321], :legend=>true, :height=>300, :width=>490, :zoom=>true}}>, #<Nyaplot::Plot:0x007fef4d454d10 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fef4d3c09a8 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :color=>"green", :title=>"1"}, :data=>"b73d2efc-1b2f-48ca-94df-6614eadafd2b"}, @xrange=[2.430278945680712, 10.997383495052622], @yrange=[2.255979669395808, 11.833176163016706]>, #<Nyaplot::Diagram:0x007fef4d48b720 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :color=>"orange", :title=>"0"}, :data=>"c7c3b045-8ef7-43e8-a2ff-3cfc3239281a"}, @xrange=[-8.636998125818076, 2.382550825493487], @yrange=[-6.599669288860291, 2.150066848382152]>, #<Nyaplot::Diagram:0x007fef4d37eb20 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"ERR 0.00%", :color=>"blue"}, :data=>"81774c37-eb34-4b06-96f5-c2577a1061d9"}, @xrange=[-18, 24], @yrange=[-24, 18]>, #<Nyaplot::Diagram:0x007fef4bf085d8 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"ERR 0.00%", :color=>"red"}, :data=>"d684dc5a-29f3-450b-af8c-f3fd4f18f080"}, @xrange=[-18, 24], @yrange=[-21, 16]>], :options=>{:x_label=>"Variance: 10", :y_label=>"", :xrange=>[-13.636998125818076, 20.997383495052624], :yrange=>[-11.59966928886029, 21.833176163016706], :legend=>true, :height=>300, :width=>490, :zoom=>true}}>, #<Nyaplot::Plot:0x007fef4bf04c30 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fef4b42a698 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :color=>"green", :title=>"1"}, :data=>"6db15855-d1f1-4d5e-8cfe-bbfbacf21617"}, @xrange=[-4.87891345140258, 14.87961612812941], @yrange=[-8.626117503699154, 13.72364004667482]>, #<Nyaplot::Diagram:0x007fef4d4143c8 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :color=>"orange", :title=>"0"}, :data=>"58a55421-f53d-4569-9a5c-59e6afecfafe"}, @xrange=[-11.424250682697602, 8.594567565260967], @yrange=[-13.040396333964598, 9.78458852949383]>, #<Nyaplot::Diagram:0x007fef4bf79b48 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"ERR 23.00%", :color=>"blue"}, :data=>"8de1daa4-e8fe-463b-9fac-eb026e21803a"}, @xrange=[-21, 28], @yrange=[-28, 21]>, #<Nyaplot::Diagram:0x007fef4c184690 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"ERR 23.00%", :color=>"red"}, :data=>"f66cf417-85d8-47b9-aae6-ecebf2b35a8f"}, @xrange=[-21, 28], @yrange=[-41, 31]>], :options=>{:x_label=>"Variance: 30", :y_label=>"", :xrange=>[-16.424250682697604, 24.87961612812941], :yrange=>[-18.040396333964598, 23.72364004667482], :legend=>true, :height=>300, :width=>490, :zoom=>true}}>, #<Nyaplot::Plot:0x007fef4c1837b8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007fef4bfd50d8 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :